<a href="https://colab.research.google.com/github/AbhiJeet70/Attack_GraphNN/blob/main/Poison_Cora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/DSE-MSU/DeepRobust.git
%cd DeepRobust
!pip install gensim==3.8.3
!python setup.py install

import os
import collections
!pip install torch
import torch
!pip install torch_geometric
import torch_geometric
from torch import Tensor
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import statistics


# DATA SPLIT
import torch_geometric.transforms as T
from deeprobust.graph.data import Dataset, PrePtbDataset, Dpr2Pyg

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(Net, self).__init__()
    self.conv1 = GCNConv(input_dim, hidden_dim)
    self.conv2 = GCNConv(hidden_dim, output_dim)

  def forward(self, data):
    x, edge_index = data.x, data.edge_index

    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = F.dropout(x, training = self.training)
    x = self.conv2(x, edge_index)

    return F.log_softmax(x, dim = 1)

# You can either use setting='prognn' or seed=15 to get the prognn splits
#data = Dataset(root='/tmp/', name='cora', setting='prognn')
data = Dataset(root='/tmp/', name='cora') # load clean graph
pyg_data = Dpr2Pyg(data) # convert dpr to pyg
print('DATA')
print(pyg_data)


#average = []
#for i in range(1, 51):
model = Net(pyg_data.num_node_features, 512, pyg_data.num_classes).to(device)
data = pyg_data[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas = (0.9, 0.999), eps=1e-08, weight_decay = 0)

model.train()
for epoch in range(1,501):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print(f'Accuracy: {acc:.4f}')
#average.append(acc)
#print(average)
#print(f'Average Accuracy: {statistics.mean(average):.4f}')

In [ ]:
# Load meta attacked data with 5% perturbation
perturbed_data = PrePtbDataset(root='/tmp/', name='cora', attack_method='meta', ptb_rate=0.05)
perturbed_adj = perturbed_data.adj
pyg_data.update_edge_index(perturbed_adj) # inplace operation

print('PERTURBATED DATA')
print(pyg_data)
print(len(pyg_data))
print(pyg_data[0])

model = Net(pyg_data.num_node_features, 512, pyg_data.num_classes).to(device)
data = pyg_data[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas = (0.9, 0.999), eps=1e-08, weight_decay = 0)

model.train()
average = []
for i in range(1, 51):
  for epoch in range(1,501):
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()

  model.eval()

  _, pred = model(data).max(dim=1)
  correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
  acc = correct / data.test_mask.sum().item()
  print(f'Accuracy: {acc:.4f}')
  average.append(acc)
  print(average)
print(f'Average Accuracy: {statistics.mean(average):.4f}')

In [ ]:
# Load meta attacked data with 10% perturbation
perturbed_data = PrePtbDataset(root='/tmp/', name='cora', attack_method='meta', ptb_rate=0.10)
perturbed_adj = perturbed_data.adj
pyg_data.update_edge_index(perturbed_adj) # inplace operation

print('PERTUBATED DATA')
print(pyg_data)
print(len(pyg_data))
print(pyg_data[0])

model = Net(pyg_data.num_node_features, 512, pyg_data.num_classes).to(device)
data = pyg_data[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas = (0.9, 0.999), eps=1e-08, weight_decay = 0)

model.train()
average = []
for i in range(1, 51):
  for epoch in range(1,501):
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()

  model.eval()

  _, pred = model(data).max(dim=1)
  correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
  acc = correct / data.test_mask.sum().item()
  print(f'Accuracy: {acc:.4f}')
  average.append(acc)
  print(average)
print(f'Average Accuracy: {statistics.mean(average):.4f}')

In [ ]:
# Load meta attacked data with 15% perturbation
perturbed_data = PrePtbDataset(root='/tmp/', name='cora', attack_method='meta', ptb_rate=0.15)
perturbed_adj = perturbed_data.adj
pyg_data.update_edge_index(perturbed_adj) # inplace operation

print('PERTUBATED DATA')
print(pyg_data)
print(len(pyg_data))
print(pyg_data[0])

model = Net(pyg_data.num_node_features, 512, pyg_data.num_classes).to(device)
data = pyg_data[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas = (0.9, 0.999), eps=1e-08, weight_decay = 0)

model.train()
average = []
for i in range(1, 51):
  for epoch in range(1,501):
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()

  model.eval()

  _, pred = model(data).max(dim=1)
  correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
  acc = correct / data.test_mask.sum().item()
  print(f'Accuracy: {acc:.4f}')
  average.append(acc)
  print(average)
print(f'Average Accuracy: {statistics.mean(average):.4f}')

In [ ]:
# Load meta attacked data with 20% perturbation
perturbed_data = PrePtbDataset(root='/tmp/', name='cora', attack_method='meta', ptb_rate=0.20)
perturbed_adj = perturbed_data.adj
pyg_data.update_edge_index(perturbed_adj) # inplace operation

print('PERTUBATED DATA')
print(pyg_data)
print(len(pyg_data))
print(pyg_data[0])

model = Net(pyg_data.num_node_features, 512, pyg_data.num_classes).to(device)
data = pyg_data[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas = (0.9, 0.999), eps=1e-08, weight_decay = 0)

model.train()
average = []
for i in range(1, 51):
  for epoch in range(1,501):
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()

  model.eval()

  _, pred = model(data).max(dim=1)
  correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
  acc = correct / data.test_mask.sum().item()
  print(f'Accuracy: {acc:.4f}')
  average.append(acc)
  print(average)
print(f'Average Accuracy: {statistics.mean(average):.4f}')

In [ ]:
# Load meta attacked data with 25% perturbation
perturbed_data = PrePtbDataset(root='/tmp/', name='cora', attack_method='meta', ptb_rate=0.25)
perturbed_adj = perturbed_data.adj
pyg_data.update_edge_index(perturbed_adj) # inplace operation

print('PERTUBATED DATA')
print(pyg_data)
print(len(pyg_data))
print(pyg_data[0])

model = Net(pyg_data.num_node_features, 512, pyg_data.num_classes).to(device)
data = pyg_data[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas = (0.9, 0.999), eps=1e-08, weight_decay = 0)

model.train()
average = []
for i in range(1, 51):
  for epoch in range(1,501):
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()

  model.eval()

  _, pred = model(data).max(dim=1)
  correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
  acc = correct / data.test_mask.sum().item()
  print(f'Accuracy: {acc:.4f}')
  average.append(acc)
  print(average)
print(f'Average Accuracy: {statistics.mean(average):.4f}')